<a href="https://colab.research.google.com/github/gmazzitelli/GSN2/blob/master/MissioniDotazioni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**per poter essere lle il file in questo modo deve essere un google speradsheet**

quindi:
1. salvare i dati delle missioni come file excell
2. andare nella directory dove si vuole salvare il file
3. aprire un nuovo file google spreadsheat
*3.bis 2 e 3 si possono saltare se e inserire nel file missioni esistenti con import.*
4. fare import dal menu del file
5. attivare lo shearing (anche solo in visione)




In [0]:
import numpy as np
import pandas as pd
url_csv_file = "https://docs.google.com/spreadsheet/ccc?key=1l-nUkzMRyt4gte4nkMcoBBzDZElnOIeBsGA8zIOVP_E&output=csv"
df = pd.read_csv(url_csv_file)

In [2]:
df

,Data,Tipo,Num,Doc,Intestatario,Descrizione,Assegn.,SubJudice,Iniziale,Variaz.,Totale
0,07/01/2019,NaN,NaN,Stanziamento iniziale (assegnazioni),NaN,Stanziamento iniziale,"34.000,00","0,00","0,00","0,00","34.000,00"
1,16/11/2018,Missioni,29001.0,Impegno contabile,MARTINI AGNESE,Laboratori Nazionali del Sud \ riunione IB dat...,"0,00","0,00","7,65","0,00","7,65"
2,07/01/2019,Missioni,29181.0,Impegno contabile,RICCI MARCO,Laboratori Nazionali di Legnaro \ LNL 10-11 e ...,"0,00","0,00","218,80","0,00","218,80"
3,08/01/2019,Missioni,29209.0,Impegno contabile,RICCI MARCO,CERN \ Presentazione al CERN Recognized Experi...,"0,00","0,00","770,70","0,00","770,70"
4,14/01/2019,Missioni,29267.0,Impegno contabile,FRANCESCHI MASSIMO ALBERTO,Laboratori Nazionali del Gran Sasso \ Meeting ...,"0,00","0,00","183,29","0,00","183,29"
5,15/01/2019,Missioni,29273.0,Impegno contabile,NAPOLITANO TOMMASO,Laboratori Nazionali del Gran Sasso \ Partecip...,"0,00","0,00","198,00","0,00","198,00"
6,21/01/2019,Missioni,29322.0,Impegno contabile,RICCI MARCO,Trento Institute for Fundamental Physics and A...,"0,00","0,00","475,70","0,00","475,70"
7,28/01/2019,Missioni,29368.0,Impegno contabile,FRANCESCHI MASSIMO ALBERTO,Torrita di Siena \ Test di vibrazione esperime...,"0,00","0,00","494,55","0,00","494,55"
8,05/02/2019,Missioni,29431.0,Impegno contabile,NAPOLITANO TOMMASO,Torrita di Siena (SI) \ TEST DI VIBRAZIONE ESP...,"0,00","0,00","499,00","0,00","499,00"
9,24/02/2019,Missioni,29560.0,Impegno contabile,RICCI MARCO,Sezione di Roma Tor Vergata \ Riunione analisi...,"0,00","0,00","39,30","0,00","39,30"


In [3]:
assegnazione = df[df.Doc=="Stanziamento iniziale (assegnazioni)"].Totale.str.replace('.', '').str.replace(',', '.').astype(float).sum()
speso = df[(df.Doc!="Stanziamento iniziale (assegnazioni)")].Totale.str.replace('.', '').str.replace(',', '.').astype(float).sum()
print ("Assegnato = ", assegnazione, "Speso = ", speso, "Disponibile = ", assegnazione-speso)
SpesoT=0.

Assegnato =  34000.0 Speso =  19611.589999999997 Disponibile =  14388.410000000003


In [4]:
df = df.drop(df.index[0])
df.Intestatario.value_counts()

RICCI MARCO                   27
NAPOLITANO TOMMASO            14
FRANCESCHI MASSIMO ALBERTO     9
PAOLONI ALESSANDRO             5
MAZZITELLI GIOVANNI            4
DE PAOLIS ALBERTO              2
LIGI CARLO                     2
MARTINI AGNESE                 2
Capone Antonio                 1
DI GIOVANNI GIANNI             1
Name: Intestatario, dtype: int64

In [5]:
Limadou = 2000.0
JemEUSO = 15000.0
assegnato = Limadou+JemEUSO
user0 = (df.Intestatario=="RICCI MARCO") ^ (df.Intestatario=="DE PAOLIS ALBERTO")
speso = df[user0].Totale.str.replace('.', '').str.replace(',', '.').astype(float).sum()
print ("Speso Missioni = ",  speso, "Assegnato = ", assegnato, "Rimanete = ", assegnato-speso)
SpesoT+=speso
df[user0]

Speso Missioni =  9390.000000000002 Assegnato =  17000.0 Rimanete =  7609.999999999998


,Data,Tipo,Num,Doc,Intestatario,Descrizione,Assegn.,SubJudice,Iniziale,Variaz.,Totale
2,07/01/2019,Missioni,29181.0,Impegno contabile,RICCI MARCO,Laboratori Nazionali di Legnaro \ LNL 10-11 e ...,"0,00","0,00","218,80","0,00","218,80"
3,08/01/2019,Missioni,29209.0,Impegno contabile,RICCI MARCO,CERN \ Presentazione al CERN Recognized Experi...,"0,00","0,00","770,70","0,00","770,70"
6,21/01/2019,Missioni,29322.0,Impegno contabile,RICCI MARCO,Trento Institute for Fundamental Physics and A...,"0,00","0,00","475,70","0,00","475,70"
9,24/02/2019,Missioni,29560.0,Impegno contabile,RICCI MARCO,Sezione di Roma Tor Vergata \ Riunione analisi...,"0,00","0,00","39,30","0,00","39,30"
10,28/02/2019,Missioni,29603.0,Impegno contabile,RICCI MARCO,Sezione di Roma Tor Vergata \ Esecuzione test ...,"0,00","0,00","67,90","0,00","67,90"
16,26/03/2019,Missioni,29805.0,Impegno contabile,RICCI MARCO,Servizio di Presidenza \ Incontro con Presiden...,"0,00","0,00","18,00","0,00","18,00"
17,03/04/2019,Missioni,29882.0,Impegno contabile,RICCI MARCO,Siena \ Partecipazione con presentazione alla ...,"0,00","0,00","365,95","0,00","365,95"
20,30/04/2019,Missioni,30045.0,Impegno contabile,RICCI MARCO,Sezione di Roma Tor Vergata \ Riunione analisi...,"0,00","0,00","19,30","0,00","19,30"
21,13/05/2019,Missioni,30146.0,Impegno contabile,RICCI MARCO,LODZ \ Partecipazione a General Collaboration ...,"0,00","0,00","1.434,20","0,00","1.434,20"
23,28/05/2019,Missioni,30277.0,Impegno contabile,DE PAOLIS ALBERTO,Torrita di Siena \ Test Vibrazione Esperimento...,"0,00","0,00","273,39","0,00","273,39"


In [6]:
Darkside = 5000.0
assegnato = Darkside
user1 = (df.Intestatario=="NAPOLITANO TOMMASO") ^ (df.Intestatario=="FRANCESCHI MASSIMO ALBERTO")
speso = df[user1].Totale.str.replace('.', '').str.replace(',', '.').astype(float).sum()
print ("Speso Missioni = ",  speso, "Assegnato = ", assegnato, "Rimanete = ", assegnato-speso)
SpesoT+=speso
df[user1]


Speso Missioni =  2905.44 Assegnato =  5000.0 Rimanete =  2094.56


,Data,Tipo,Num,Doc,Intestatario,Descrizione,Assegn.,SubJudice,Iniziale,Variaz.,Totale
4,14/01/2019,Missioni,29267.0,Impegno contabile,FRANCESCHI MASSIMO ALBERTO,Laboratori Nazionali del Gran Sasso \ Meeting ...,"0,00","0,00","183,29","0,00","183,29"
5,15/01/2019,Missioni,29273.0,Impegno contabile,NAPOLITANO TOMMASO,Laboratori Nazionali del Gran Sasso \ Partecip...,"0,00","0,00","198,00","0,00","198,00"
7,28/01/2019,Missioni,29368.0,Impegno contabile,FRANCESCHI MASSIMO ALBERTO,Torrita di Siena \ Test di vibrazione esperime...,"0,00","0,00","494,55","0,00","494,55"
8,05/02/2019,Missioni,29431.0,Impegno contabile,NAPOLITANO TOMMASO,Torrita di Siena (SI) \ TEST DI VIBRAZIONE ESP...,"0,00","0,00","499,00","0,00","499,00"
12,11/03/2019,Missioni,29677.0,Impegno contabile,FRANCESCHI MASSIMO ALBERTO,L'Aquila \ General meeting esperimento DARKSID...,"0,00","0,00","351,60","0,00","351,60"
13,11/03/2019,Missioni,29678.0,Impegno contabile,FRANCESCHI MASSIMO ALBERTO,L'Aquila \ Technical Board esperimento DARKSID...,"0,00","0,00","176,60","0,00","176,60"
14,11/03/2019,Missioni,29683.0,Impegno contabile,NAPOLITANO TOMMASO,GSSI L'Aquila \ Partecipazione a General Meeti...,"0,00","0,00","376,00","0,00","376,00"
15,11/03/2019,Missioni,29684.0,Impegno contabile,NAPOLITANO TOMMASO,GSSI L'Aquila \ Partecipazione a Technical Boa...,"0,00","0,00","200,00","0,00","200,00"
18,10/04/2019,Missioni,29927.0,Impegno contabile,NAPOLITANO TOMMASO,CERN \ Meeting Tecnico su Integrazione Esperim...,"0,00","0,00","1.105,89","0,00","1.105,89"
24,28/05/2019,Missioni,30278.0,Impegno contabile,NAPOLITANO TOMMASO,Torrita di Siena (SI) \ TEST DI VIBRAZIONE ESP...,"0,00","0,00","289,00","0,00","289,00"


In [7]:
KM3 = 3500.0
assegnato = KM3
user2 = (df.Intestatario=="MARTINI AGNESE") ^ (df.Intestatario=="Capone Antonio")
speso = df[user2].Totale.str.replace('.', '').str.replace(',', '.').astype(float).sum()
print ("Speso Missioni = ",  speso, "Assegnato = ", assegnato, "Rimanete = ", assegnato-speso)
SpesoT+=speso
df[user2]


Speso Missioni =  2665.65 Assegnato =  3500.0 Rimanete =  834.3499999999999


,Data,Tipo,Num,Doc,Intestatario,Descrizione,Assegn.,SubJudice,Iniziale,Variaz.,Totale
1,16/11/2018,Missioni,29001.0,Impegno contabile,MARTINI AGNESE,Laboratori Nazionali del Sud \ riunione IB dat...,"0,00","0,00","7,65","0,00","7,65"
28,08/07/2019,Missioni,30624.0,Impegno contabile,MARTINI AGNESE,Varsavia \ collaboration meeting KM3NET data d...,"0,00","0,00","1.470,00","0,00","1.470,00"
31,11/07/2019,Missioni,30667.0,Impegno contabile,Capone Antonio,Varsavia \ partecipazione alle riunioni degli ...,"0,00","0,00","1.188,00","0,00","1.188,00"


In [8]:
DTZ = 8500.0
assegnato = DTZ
user3 = np.invert(user0 ^ user1 ^ user2)
speso = df[user3].Totale.str.replace('.', '').str.replace(',', '.').astype(float).sum()
print ("Speso Missioni = ",  speso, "Assegnato = ", assegnato, "Rimanete = ", assegnato-speso)
SpesoT+=speso
df[user3]


Speso Missioni =  4650.5 Assegnato =  8500.0 Rimanete =  3849.5


,Data,Tipo,Num,Doc,Intestatario,Descrizione,Assegn.,SubJudice,Iniziale,Variaz.,Totale
11,07/03/2019,Missioni,29660.0,Impegno contabile,PAOLONI ALESSANDRO,Siena \ Riunione della commissione scientifica...,"0,00","0,00","680,00","0,00","680,00"
19,29/04/2019,Missioni,30035.0,Impegno contabile,MAZZITELLI GIOVANNI,Palma de Mallorca \ presentazioni alla confere...,"0,00","0,00","527,79","0,00","527,79"
22,13/05/2019,Missioni,30154.0,Impegno contabile,LIGI CARLO,Friburgo \ partecipazione a conferenza PATRAS ...,"0,00","0,00","1.000,00","0,00","1.000,00"
30,10/07/2019,Missioni,30652.0,Impegno contabile,DI GIOVANNI GIANNI,CERN \ Open Day Cern data da 13-SEP-19 a 15-SE...,"0,00","0,00","215,00","0,00","215,00"
33,15/07/2019,Missioni,30688.0,Impegno contabile,PAOLONI ALESSANDRO,Laboratori Nazionali del Gran Sasso \ Partecip...,"0,00","0,00","154,67","0,00","154,67"
34,10/08/2019,Missioni,30813.0,Impegno contabile,MAZZITELLI GIOVANNI,Alghero \ ILS - Innovative learning School dat...,"0,00","0,00","350,00","0,00","350,00"
35,10/08/2019,Missioni,30814.0,Impegno contabile,MAZZITELLI GIOVANNI,Milano \ Referaggio Nu @ FNAL / ICARUS data da...,"0,00","0,00","600,00","0,00","600,00"
36,12/08/2019,Missioni,30815.0,Impegno contabile,PAOLONI ALESSANDRO,Milano \ Riunione per referaggio esperimenti I...,"0,00","0,00","500,00","0,00","500,00"
37,22/08/2019,Missioni,30834.0,Impegno contabile,MAZZITELLI GIOVANNI,Torino \ RIunione CSN2 data da 16-SEP-19 a 20-...,"0,00","0,00","1.170,00","0,00","1.170,00"
49,07/03/2019,Missioni,29660.0,Variazione impegno,PAOLONI ALESSANDRO,Siena \ Riunione della commissione scientifica...,"0,00","0,00","0,00","-506,70","-506,70"


In [9]:
print ("Assegnato = ", DTZ+KM3+Darkside+Limadou+JemEUSO, "Speso = ", SpesoT, "Disponibile = ", DTZ+KM3+Darkside+Limadou+JemEUSO-SpesoT)


Assegnato =  34000.0 Speso =  19611.590000000004 Disponibile =  14388.409999999996
